In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
tabla_completa = pd.read_csv('events_up_to_01062018.csv');
tabla_completa.info()

In [ ]:
res = pd.read_csv('labels_training_set.csv');

In [ ]:
tabla_completa['aux']=1
#Voy a contar la cantidad para cada tipo de evento de cada persona
num_eventos = tabla_completa.groupby(['person','event'])['aux'].sum()

num_eventos = num_eventos.unstack().reset_index().fillna(0)
num_eventos = num_eventos.set_index('person')
num_eventos['total_eventos'] = num_eventos.sum(axis=1)
#normalizo los numeros obtenidos según la cantidad de eventos totales de esa persona
num_eventos_norm = num_eventos.loc[:,"ad campaign hit":"visited site"].div(num_eventos["total_eventos"], axis=0)
#num_eventos.sort_values(by='conversion',ascending=False)
#num_eventos_norm.sort_values(by='conversion',ascending=False)

In [ ]:
tabla_completa['timestamp'] = pd.to_datetime(tabla_completa['timestamp'])
#tabla_completa['timestamp'].max() -tabla_completa['timestamp'].min()

time_for_user = tabla_completa.groupby(['person'])['timestamp'].apply(lambda x: x.max() - x.min())
time_for_user = time_for_user.reset_index()
time_for_user['total_horas'] = time_for_user['timestamp'].dt.total_seconds()/3600
time_for_user = time_for_user.drop(['timestamp'],axis='columns')

In [ ]:
num_eventos_norm = num_eventos_norm.reset_index()
num_eventos = num_eventos.reset_index()
total_eventos = pd.merge(num_eventos,num_eventos_norm, on='person',how='inner')
total_eventos = pd.merge(total_eventos,time_for_user,on='person',how='inner')

In [ ]:
intento_f = pd.merge(total_eventos,res,on='person',how='inner')
intento_f = intento_f.set_index('person')
intento_f.head()

In [ ]:
x, y = intento_f.iloc[:,:-1],intento_f.iloc[:,-1]

In [ ]:
test_size = 0.2
random_state = 10

In [ ]:
datos_train, datos_test, cat_train, cat_test = train_test_split(x, y, test_size = test_size, random_state = random_state)

In [ ]:
gnb = GaussianNB()

In [ ]:
gnb.fit(datos_train,cat_train)

In [ ]:
cat_pred = gnb.predict(datos_test)

print('accuracy: {0:.2f}%'.format(accuracy_score(cat_test,cat_pred)*100))

In [ ]:
rmse = np.sqrt(mean_squared_error(cat_test, cat_pred))
print("RMSE: %f" % (rmse))

In [ ]:
final = pd.read_csv('trocafone_kaggle_test.csv')
final = pd.merge(total_eventos,final,on='person',how='inner')
final = final.set_index('person')
x_entrega = final

In [ ]:
entrega = gnb.predict(x_entrega)
y_summit = pd.DataFrame(entrega)
y_summit.columns = ['label']
y_summit['person'] = x_entrega.reset_index()['person']
y_person = x_entrega.reset_index()['person']

y_summit = y_summit[['person','label']]
y_summit.to_csv('summit09.csv', index=False)

In [ ]:
y_summit.max()